## 신용카드 사기 거래 탐지 AI 경진대회

- 참가자: 김승우, 정연호, 이경선
- 목표: 다양한 방법론을 적용하고 그 중에서 가장 정확도가 높은 방법 찾기.

In [22]:
import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from pandas import Series, DataFrame

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
train_df = pd.read_csv('C:/Users/김승우/Desktop/파이썬/train.csv')
train_df = train_df.drop(columns=['ID'])
val_df = pd.read_csv('C:/Users/김승우/Desktop/파이썬/val.csv')
val_df = val_df.drop(columns=['ID'])
val_re=val_df.drop(columns=['Class'])

In [5]:
val_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,...,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,-0.255991,-0.994878,0
1,0.962496,0.328461,-0.171479,2.109204,1.129566,1.696038,0.107712,0.521502,-1.191311,0.724396,...,0.402492,-0.048508,-1.371866,0.390814,0.199964,0.016371,-0.014605,0.168937,-0.994784,0
2,1.145524,0.575068,0.194008,2.598192,-0.092210,-1.044430,0.531588,-0.241888,-0.896287,0.757952,...,-0.119703,-0.076510,0.691320,0.633984,0.048741,-0.053192,0.016251,0.169496,-0.994502,0
3,0.927060,-0.323684,0.387585,0.544474,0.246787,1.650358,-0.427576,0.615371,0.226278,-0.225495,...,0.079359,0.096632,-0.992569,0.085096,0.377447,0.036096,-0.005960,0.331307,-0.994467,0
4,-3.005237,2.600138,1.483691,-2.418473,0.306326,-0.824575,2.065426,-1.829347,4.009259,6.051521,...,-0.181268,-0.163747,0.515821,0.136318,0.460054,-0.251259,-1.105751,-0.287012,-0.994373,0


In [39]:
val_fraud=val_df[val_df["Class"]==1]
print(val_fraud)
index=val_fraud.index.to_list()
print("\n Index info")
print(index)

              V1         V2         V3         V4         V5        V6  \
71     -3.043541  -3.157307   1.088463   2.288644   1.359805 -1.064823   
641     0.316459   3.809076  -5.615159   6.047445   1.554026 -2.651353   
677     0.857321   4.093912  -7.423894   7.380245   0.973366 -2.730762   
836    -1.426623   4.141986  -9.804103   6.666273  -4.749527 -2.073129   
1196  -16.367923   9.223692 -23.270631  11.844777  -9.462037 -2.450444   
1201  -16.917468   9.669900 -23.736443  11.824990  -9.830548 -2.514829   
1547  -18.474868  11.586381 -21.402917   6.038515 -14.451158 -4.146524   
3055   -3.896583   4.518355  -4.454027   5.547453  -4.121459 -1.163407   
4039    1.159373   2.844795  -4.050680   4.777701   2.948980 -2.010361   
4267  -13.897206   6.344280 -14.281666   5.581009 -12.887133 -3.146176   
4396  -19.139733   9.286847 -20.134992   7.818673 -15.652208 -1.668348   
7702   -2.335655   2.225380  -3.379450   2.178538  -3.568264  0.316814   
9326    1.080433   0.962831  -0.278065

In [31]:
import pandas as pd
import numpy as np

def mean_norm(df_input):
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

val_df_ss = mean_norm(val_df)
print(val_df_ss)

             V1        V2        V3        V4        V5        V6        V7  \
0     -0.177780  0.696225  0.695418 -0.159472  0.376984 -0.177240  0.554467   
1      0.495965  0.203369 -0.115281  1.499718  0.848996  1.326225  0.087678   
2      0.590767  0.356999  0.128418  1.847717 -0.066091 -0.794527  0.451478   
3      0.477610 -0.202902  0.257491  0.386141  0.187812  1.290875 -0.371745   
4     -1.559174  1.618569  0.988350 -1.722510  0.232405 -0.624389  1.767928   
...         ...       ...       ...       ...       ...       ...       ...   
28457 -0.285576  0.892088 -0.209812  0.657694  0.394232 -0.424031  0.910213   
28458 -0.398567  0.365291 -0.606110 -0.299429  0.678200 -0.575038  0.646269   
28459 -0.234077  0.809992 -0.113350  0.697216  0.436601 -0.454964  1.071016   
28460  1.053844 -0.110421 -0.798960  0.165597 -0.003554 -0.548548  0.009865   
28461 -0.127112  0.329223  0.467476  0.489565 -0.280113  0.496386 -0.593697   

             V8        V9       V10  ...       V22 

In [44]:
val_df_ss_fraud=val_df_ss.loc[index]
print(val_df_ss_fraud)

              V1         V2         V3        V4         V5        V6  \
71     -1.579014  -1.968181   0.724821  1.627420   1.021441 -0.810309   
641     0.161341   2.371708  -3.745009  4.302460   1.166909 -2.038070   
677     0.441488   2.549154  -4.951034  5.250978   0.732005 -2.099521   
836    -0.741511   2.579103  -6.538107  4.742863  -3.554333 -1.590602   
1196   -8.480551   5.744884 -15.517296  8.428267  -7.083914 -1.882594   
1201   -8.765195   6.022861 -15.827890  8.414186  -7.359922 -1.932419   
1547   -9.571870   7.216782 -14.271945  4.296105 -10.820672 -3.195130   
3055   -2.020858   2.813572  -2.970792  3.946629  -3.083922 -0.886600   
4039    0.597940   1.770984  -2.701848  3.398816   2.211703 -1.542028   
4267   -7.200811   3.951080  -9.523649  3.970510  -9.649247 -2.420996   
4396   -9.916246   5.784228 -13.426520  5.562996 -11.720235 -1.277356   
7702   -1.212355   1.385103  -2.254287  1.549061  -2.669590  0.258891   
9326    0.557052   0.598566  -0.186351  1.951000   

In [46]:
val_df_ss_fraud=abs(val_df_ss_fraud)

In [47]:
val_df_ss_fraud.sum()

V1       109.132803
V2        94.358654
V3       199.055334
V4       117.180102
V5       121.212697
V6        44.029972
V7       204.621073
V8        83.495298
V9        92.732229
V10      209.471239
V11      132.041924
V12      222.479738
V13       29.074367
V14      251.666041
V15       31.992814
V16      168.733092
V17      296.792380
V18      116.963690
V19       50.857020
V20       35.710556
V21       70.708639
V22       31.490314
V23       30.823325
V24       21.567701
V25       29.807588
V26       26.109745
V27      103.371428
V28       44.620707
V29       12.632522
V30       28.926024
Class    923.542111
dtype: float64

In [36]:
for i in 

1.5790139747048917

In [28]:
abs(val_df_ss["V1"][71])

array([[-0.1777828 ,  0.69623726,  0.69543061, ..., -0.35250264,
        -1.99699312, -0.03248306],
       [ 0.49597343,  0.20337215, -0.11528319, ..., -0.22556347,
        -1.99682475, -0.03248306],
       [ 0.59077699,  0.35700505,  0.12842026, ..., -0.2253965 ,
        -1.99631967, -0.03248306],
       ...,
       [-0.23408074,  0.8100065 , -0.11335212, ..., -0.11532133,
         1.63881874, -0.03248306],
       [ 1.05386235, -0.1104227 , -0.79897392, ..., -0.3566769 ,
         1.63911338, -0.03248306],
       [-0.12711389,  0.32922889,  0.4674842 , ..., -0.32612133,
         1.63919756, -0.03248306]])

array([-0.1777828 ,  0.69623726,  0.69543061, -0.15947447,  0.376991  ,
       -0.17724262,  0.55447691,  0.05483876, -0.67088999, -0.33612993,
        0.99716533,  0.84172605,  1.00213113, -0.48556967,  0.16092449,
        0.84989266, -0.66770914,  0.56517958,  0.55833357,  0.27657855,
       -0.35232708, -0.88027719, -0.2079428 , -0.64062346, -0.13669328,
        0.19323453,  0.62587991,  0.27389654, -0.35250264, -1.99699312,
       -0.03248306])

             V1        V2        V3        V4        V5        V6        V7  \
0     -0.177780  0.696225  0.695418 -0.159472  0.376984 -0.177240  0.554467   
1      0.495965  0.203369 -0.115281  1.499718  0.848996  1.326225  0.087678   
2      0.590767  0.356999  0.128418  1.847717 -0.066091 -0.794527  0.451478   
3      0.477610 -0.202902  0.257491  0.386141  0.187812  1.290875 -0.371745   
4     -1.559174  1.618569  0.988350 -1.722510  0.232405 -0.624389  1.767928   
...         ...       ...       ...       ...       ...       ...       ...   
28457 -0.285576  0.892088 -0.209812  0.657694  0.394232 -0.424031  0.910213   
28458 -0.398567  0.365291 -0.606110 -0.299429  0.678200 -0.575038  0.646269   
28459 -0.234077  0.809992 -0.113350  0.697216  0.436601 -0.454964  1.071016   
28460  1.053844 -0.110421 -0.798960  0.165597 -0.003554 -0.548548  0.009865   
28461 -0.127112  0.329223  0.467476  0.489565 -0.280113  0.496386 -0.593697   

             V8        V9       V10  ...       V22 